In [2]:
import pymongocrypt
import pymongo
from pymongo import MongoClient
from pymongo.encryption_options import AutoEncryptionOpts
from pymongo.encryption import ClientEncryption
import base64
from bson.codec_options import CodecOptions
from bson.binary import STANDARD, UUID
from pprint import pprint
import os
from pprint import pprint
from secrets import token_bytes

#https://docs.mongodb.com/drivers/security/client-side-field-level-encryption-guide/#introduction
#https://github.com/mongodb-university/csfle-guides/tree/master/python

In [34]:
path = "master-key.txt"
file_bytes = os.urandom(96)
with open(path, "wb") as f:
  f.write(file_bytes)

path = "./master-key.txt"
with open(path, "rb") as f:
  local_master_key = f.read()

kms_providers = {
  "local": {
    "key": local_master_key # local_master_key variable from the previous step
  },
}

connection_string = "mongodb://localhost:27017"
key_vault_namespace = "encryption.__keyVault"
client = MongoClient(connection_string)
client_encryption = ClientEncryption(
    kms_providers, # pass in the kms_providers variable from the previous step
    key_vault_namespace,
    client,
    CodecOptions(uuid_representation=STANDARD),
)

data_key_id = client_encryption.create_data_key("local", key_alt_names=["example"])

In [35]:
connection_string = "mongodb://localhost:27017"
key_vault_db = "encryption"
key_vault_coll = "__keyVault"
client = MongoClient(connection_string)
key_vault = client[key_vault_db][key_vault_coll]
key = key_vault.find_one({"_id": data_key_id})
pprint(key) #pprint(key['_id'])

{'_id': UUID('dafd3d10-0849-4663-b9ae-38b11cc689bd'),
 'creationDate': datetime.datetime(2022, 3, 19, 2, 7, 34, 197000),
 'keyAltNames': ['example'],
 'keyMaterial': b'\xda\x97\xab\x081\xa7\x08\xbcs\xf6\xbdL\xb1r\xb6O'
                b'\x10\x94\x91*\xd2\x8f\xea\xf2\xd0%@C\t;\x8f\xda\xb8L\xfd\x0c'
                b'3\x98"\xc3L\xe9+J\x9e\xe81Q\x8e\xa2\xbcwm\x8daN'
                b'\x84\xa9\xff\xb2\xf4\xa5\x9b\xa9\xf2\x11=\xae\x18\xba\xb0\x1f'
                b'\x9aRx"\x9bg@\xd3\xbb3k\x84\x7fm\xf9^\x94*\xb9\xe8'
                b'\xd6\x8e\x0c\x12\xe5\xc2\x94\x117\xd5G\ts\xd9\x94"'
                b'\xcd\xecL\xcc\xef\xedW\xf2F\xc9\xf7)\xb2;\x97\x9b\xedX\xf0k'
                b'\xe8\xc4\x8b:|K/\x11h\x1f\xc3\x89G\x13\x89\xe1\xa5\xbd!:'
                b'\xdcD(U\xab\xf3K\xd9\xd8\x8eMU',
 'masterKey': {'provider': 'local'},
 'status': 0,
 'updateDate': datetime.datetime(2022, 3, 19, 2, 7, 34, 197000)}


In [68]:
#works for 1 field
schema = {
    "bsonType": "object",
    "properties": {
        "ssn": {
            "encrypt": {
                "bsonType": "string",
                "algorithm": "AEAD_AES_256_CBC_HMAC_SHA_512-Random",
                "keyId": [key['_id']],  # Reference the key
            }
        },
    },
}

In [83]:
#dosent work.
schema = {
  "bsonType": "object",
  "encryptMetadata": {
    "keyId": [
      {
        "$binary": {
          "base64": 'dafd3d10-0849-4663-b9ae-38b11cc689bd',
          "subType": "04"
        }
      }
    ]
  },
  "properties": {
    "insurance": {
      "bsonType": "object",
      "properties": {
        "policyNumber": {
          "encrypt": {
            "bsonType": "int",
            "algorithm": "AEAD_AES_256_CBC_HMAC_SHA_512-Deterministic"
          }
        }
      }
    },
    "medicalRecords": {
      "encrypt": {
        "bsonType": "array",
        "algorithm": "AEAD_AES_256_CBC_HMAC_SHA_512-Random"
      }
    },
    "bloodType": {
      "encrypt": {
        "bsonType": "string",
        "algorithm": "AEAD_AES_256_CBC_HMAC_SHA_512-Random"
      }
    },
    "ssn": {
      "encrypt": {
        "bsonType": "int",
        "algorithm": "AEAD_AES_256_CBC_HMAC_SHA_512-Deterministic"
      }
    }
  }
}

In [86]:
#works
schema = {
  "bsonType": "object",
  "properties": {
    "insurance": {
      "bsonType": "object",
      "properties": {
        "policyNumber": {
          "encrypt": {
            "bsonType": "int",
            "algorithm": "AEAD_AES_256_CBC_HMAC_SHA_512-Deterministic",
            "keyId": [key['_id']]
          }
        }
      }
    },
    "medicalRecords": {
      "encrypt": {
        "bsonType": "array",
        "algorithm": "AEAD_AES_256_CBC_HMAC_SHA_512-Random",
        "keyId": [key['_id']],
      }
    },
    "bloodType": {
      "encrypt": {
        "bsonType": "string",
        "algorithm": "AEAD_AES_256_CBC_HMAC_SHA_512-Random",
        "keyId": [key['_id']],
      }
    },
    "ssn": {
      "encrypt": {
        "bsonType": "int",
        "algorithm": "AEAD_AES_256_CBC_HMAC_SHA_512-Deterministic",
        "keyId": [key['_id']]
      }
    }
  }
}

In [91]:
def insert_patient(collection, name, ssn, blood_type, medical_records, policy_number, provider):
  insurance = {
    'policyNumber': policy_number,
    'provider': provider
  }
  doc = {
      'name': name,
      'ssn': ssn,
      'bloodType': blood_type,
      'medicalRecords': medical_records,
      'insurance': insurance
  }
  collection.insert_one(doc)

insert_patient(client.encryption.test,'Ian',1231231234,'A',['120/40'],123,'')

In [85]:
key_vault_namespace = "encryption.__keyVault"

key_bytes = token_bytes(96)

# Configure a single, local KMS provider, with the saved key:
#kms_providers = {"local": {"key": key_bytes}}

patient_schema = {
  "medicalRecords.patients": schema
}

extra_options = {
   'mongocryptd_spawn_path': 'mongocryptd'
}

extra_options['mongocryptd_bypass_spawn'] = True

fle_opts = AutoEncryptionOpts(
   kms_providers,
   key_vault_namespace,
   schema_map={"encryption.test": schema},
)

client = MongoClient(connection_string, auto_encryption_opts=fle_opts)

client.encryption.test.insert_one(
        {
            
            "ssn": 123121234,
        })

c = client.encryption.test.find()
for x in c:
    print(x)

{'_id': ObjectId('62353b003bc051d9f65e9359'), 'ssn': '123-12-1234'}
{'_id': ObjectId('62353d5a3bc051d9f65e937d'), 'ssn': '123-12-1234'}
{'_id': ObjectId('62353eb13bc051d9f65e938a'), 'ssn': '123-12-1234'}
{'_id': ObjectId('623864fa3bc051d9f65e9397'), 'ssn': 123121234}


In [76]:
#insert to the collection without the schema.

fle_opts_no_schema = AutoEncryptionOpts(
    kms_providers,
    key_vault_namespace
)

client_no_schema = MongoClient(connection_string, auto_encryption_opts=fle_opts_no_schema)

client_no_schema.encryption.test.insert_one(
        {
            "ssn": "123-12-1234",
        })

c2 = client_no_schema.encryption.test.find()
for x in c2:
    print(x)

{'_id': ObjectId('62353b003bc051d9f65e9359'), 'ssn': '123-12-1234'}
{'_id': ObjectId('62353d5a3bc051d9f65e937d'), 'ssn': '123-12-1234'}


In [82]:
#query the encryption test database as a normal client.
uri = "localhost:27017"
client = pymongo.MongoClient(uri)
db = client.encryption
c3 = db.test.find()
for x in c3:
    print(x)

{'_id': ObjectId('62353b003bc051d9f65e9359'), 'ssn': Binary(b'\x02\xda\xfd=\x10\x08IFc\xb9\xae8\xb1\x1c\xc6\x89\xbd\x02]K\x9caZ\xedf\x85\xf0$E\x04\xaf\xf8\xb8\xcb\x92Y@\x9d\x1d\xa4\xe0o\xd2)\xd6o\xa1\xfa}h\xaa\xa6^\x9f\xb3\xef[l\x93I\xd5\x8e\xfd\xefq\x00\xb8\xe4\xd9K*\x85)\xa9\xf6\xcb\x81\x8d\x11\x96M\xab\x03)\xbe[Ib\x06"\xdc\xda08Tdz\x7f', 6)}
{'_id': ObjectId('62353d5a3bc051d9f65e937d'), 'ssn': Binary(b"\x02\xda\xfd=\x10\x08IFc\xb9\xae8\xb1\x1c\xc6\x89\xbd\x02\x04u\x9b\x97\x82\r\xb0\xc2\xd7g+W\x89\xa8\xb7\x1d\x92\xac\xf0\xf2\xbeg|cND\xfeG\xd1JW\xc0;\xdc\xf1\x8b\xcb{7U\x02\xaa\xe2\xef\x81\xbd\x06\x1c\xf02#\xbd [\x16\x8b\xc1\xca'L\xecb\x07\x02X:\x86\x9c\x81P\x15\xe7\xd8\x80\xe3\x9f\xfc\xa6p\x0f", 6)}
{'_id': ObjectId('62353eb13bc051d9f65e938a'), 'ssn': '123-12-1234'}
